In [11]:
#Imports
import pykep as pk
import pygmo as pg
import pygmo_plugins_nonfree as ppnf
from pykep.planet import jpl_lp
from pykep import epoch
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [46]:
# Define Launch and Arrival Dates
departure = int(pk.epoch_from_string("1997-Oct-13 00:00:00").mjd2000)  # Turns the date into mjd
arrival = int(pk.epoch_from_string("1997-Oct-16 00:00:00").mjd2000)    # Turns the date into mjd
#print(departure, arrival)                                             # print checks

In [51]:
# Define the problem i.e. Earth-Venus-Jupiter flyby with 1 DSM.
udp = pk.trajopt.mga_1dsm(
    seq=[jpl_lp('earth'), jpl_lp('venus'), jpl_lp('venus')],
    t0=[epoch(departure),epoch(arrival)],             # Departure window (in MJD2000)
    tof=[[190,200], [415,425]],                       # TOF for Earth→Venus, Venus→Jupiter
    vinf=[4.07, 4.25],                                # Departure hyperbolic excess speed [km/s]
    add_vinf_dep=True,                                # Objective includes v-infinity at Earth/Venus
    add_vinf_arr=False                                # Objective includes v-infinity at Venus/Jupiter
)

In [53]:
# Create Pygmo set
prob = pg.problem(udp)

# Define the Algorithm
uda = pg.de(gen=500)
archi = pg.archipelago(algo=uda, prob=udp, n=8, pop_size=20)
print("Running a Self-Adaptive Differential Evolution Algorithm .... on 16 parallel islands")
archi.evolve(30)
archi.wait()
sols = archi.get_champions_f()
idx = sols.index(min(sols))
print("Done!! Solutions found are: ", archi.get_champions_f())
udp.pretty(archi.get_champions_x()[idx])

Running a Self-Adaptive Differential Evolution Algorithm .... on 16 parallel islands
Done!! Solutions found are:  [array([4402.55648221]), array([4402.55648253]), array([4402.55648228]), array([4402.55648247]), array([4402.55648248]), array([4402.55648247]), array([7860.58687411]), array([4402.55648244])]
First Leg: earth to venus
Departure: 1997-Oct-15 23:59:59.999946 (-807.0000000006293 mjd2000) 
Duration: 194.47836586103543days
VINF: 4.0700000006530255 km/sec
DSM after 38.27934827305565 days
DSM magnitude: 66.48108754322708m/s

leg no. 2: venus to venus
Duration: 424.9999999996942days
Fly-by epoch: 1998-Apr-28 11:28:50.810339 (-612.5216341395939 mjd2000) 
Fly-by radius: 1.4368292109145913 planetary radii
DSM after 210.61418930449983 days
DSM magnitude: 266.0753940098629m/s

Arrival at venus
Arrival epoch: 1999-Jun-27 11:28:50.810313 (-187.52163413989967 mjd2000) 
Arrival Vinf: 8248.1499094826m/s
Total mission time: 1.6960393315831064 years (619.4783658607296 days)


In [6]:
# Problem details
print("Example objective values:", sols[:3])
print("Problem dimension:", prob.get_nx())
print("Bounds:", prob.get_lb(), prob.get_ub())
x = archi.get_champions_x()[idx]
print("Sample solution:", x)

Example objective values: [array([286.23795862]), array([286.23795867]), array([286.23795884])]
Problem dimension: 10
Bounds: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e+02
  1.00000000e-01  5.00000000e+01 -6.28318531e+00  1.10000000e+00
  1.00000000e-01  5.00000000e+01] [1.00000000e+03 1.00000000e+00 1.00000000e+00 2.50000000e+03
 9.00000000e-01 9.00000000e+02 6.28318531e+00 3.00000000e+01
 9.00000000e-01 9.00000000e+02]
Sample solution: [5.68632581e+02 5.99110400e-01 3.40396164e-01 2.50000000e+03
 8.25212248e-01 7.23895714e+02 3.05295586e+00 3.00000000e+01
 8.14500613e-01 7.52806560e+02]
